In [27]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    
    scores = []

    a_start,a_step,a_num = 0.00100,0.0000095,150

    params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

    if 1: 
        pool = Pool(processes=1)
        ls   = pool.map( one_loop1, params )


    if 0:
        ls = [ one_loop(param) for param in  params ]


    ##------------------------------------------------

    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels
var1 = 0
def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     if var1 != 0:
#         df = submission.merge(hits,  on=['hit_id'], how='left')
#     else:
#         df = hits
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

#         if len(df1) == 0:
#             continue
        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

#         print(df1.head())
        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
def trackML(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    mm = 1
    ls = []
    print(Niter)
    Niter = Niter.astype(np.int32)
    print(Niter)
    print(w1, w2, w3, Niter)
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0 + mm*(rt+0.000005*rt**2)/1000*(ii)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, x/y, x/rt , y/rt]))
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, x/y]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3,w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [7]:
def trackML2(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [21]:
def trackML3(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    y1 = y/rt
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    z4 = np.sqrt(abs(z/rt))
    
#     zz4 = np.sqrt(abs(z/r))*np.sign(z)
    
    zz4 = (r * 4 * np.pi * np.pi)/ (((2 * np.pi * r) ** 2) + (z ** 2))/1000
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    theta_deg = np.degrees(np.arctan2(rt, z))
    
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+0.0000145*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        a2 = a0 - np.nan_to_num(np.arccos(mm*ii*rt))
        saa2 = np.sin(a2)
        caa2 = np.cos(a2)
                
        ctt = np.cos((theta_deg+a1)/2)
        stt = np.sin((theta_deg+a1)/2)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, z4,
                                                           zz4, caa2, saa2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w12]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035+ii*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [22]:
######################## 
### working function ###
# ########################
# trackML <- function(dfh,w1,w2,w3,Niter){
#   dfh[,s1:=hit_id]
#   dfh[,N1:=1L] 
#   dfh[,r:=sqrt(x*x+y*y+z*z)]
#   dfh[,rt:=sqrt(x*x+y*y)]
#   dfh[,a0:=atan2(y,x)]
#   dfh[,z1:=z/rt]
#   dfh[,z2:=z/r]
#   mm     <-  1
#   for (ii in 0:Niter) {
#     mm <- mm*(-1)
#     dfh[,a1:=a0+mm*(rt+0.000005*rt^2)/1000*(ii/2)/180*pi]
#     dfh[,sina1:=sin(a1)]
#     dfh[,cosa1:=cos(a1)]
#     dfs=scale(dfh[,.(sina1,cosa1,z1,z2)])
# 	cx <- c(w1,w1,w2,w3)
#     for (jj in 1:ncol(dfs)) dfs[,jj] <- dfs[,jj]*cx[jj]
#     res=dbscan(dfs,eps=0.0035,minPts = 1)
#     dfh[,s2:=res$cluster]
#     dfh[,N2:=.N, by=s2]
#     maxs1 <- max(dfh$s1)
#     dfh[,s1:=ifelse(N2>N1 & N2<20,s2+maxs1,s1)]
#     dfh[,s1:=as.integer(as.factor(s1))]
#     dfh[,N1:=.N, by=s1]    
#   }
#   return(dfh$s1)
# }

In [23]:
def Fun4BO(w1,w2,w3,Niter):
    track_id = trackML(hits, w1,w2,w3,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [24]:
def Fun4BO2(w12):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 0.0318 # z4
    w11 = 0.000414
    Niter=179
    Niter=247
#     Niter = Niter.astype(np.int32)
    print(w12)
    track_id = trackML3(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [25]:
#######################################
# function for Bayessian Optimization #
#   (needs lists: Score and Pred)     #
#######################################
# Fun4BO <- function(w1,w2,w3,Niter) { 
#    dfh$s1 <- trackML(dfh,w1,w2,w3,Niter)
#    sub=data.table(event_id=nev,hit_id=dfh$hit_id,track_id=dfh$s1)
#    sc <- score(sub,dft)
#    list(Score=sc,Pred=0)
# }

In [34]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = [0.1, 0.2] # 0.1470 -> 0.55690
    w4 = 0.1470
    w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    w6 = [0.000001, 0.00001]
    
    Niter = 179
    Niter = 247
#     w5 = 0.01
#     for w5 in [0.012, 0.0121, 0.0122, 0.0123, 0.0124, 0.0125, 0.0126, 0.0127, 0.0128, 0.0129, 0.03]:
#         print(w5)
#         Fun4BO2(w5)
#     w10 = [0.01, 1.2]
    for w12 in np.arange(0.0003, 0.00055, 0.00005):
        print(w12)
        Fun4BO2(w12)
#     Niter = [240, 480]
#     w12 = [0.000005,0.008]
#     bo = BayesianOptimization(Fun4BO2,pbounds = {'w12':w12})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# y/r: 0.001: 0.57172
# y/r: 22 | 07m19s |    0.57196 |    0.0012 | 
# y/r: 3 | 07m10s |    0.57199 |    0.0006 | 
# y/r: 0.57204, 0.0003864
# x/r,y/r,z3: 0.572165497081175
# ctt, stt: 21 | 07m48s |    0.57217 |    0.0001 | 
# ctt, stt, theta-a1: 21 | 08m25s |    0.57217 |    0.0000 | (1e-5)
# (theta+a1)/2: 0.57217 all accross
# y3: 0.0012: (0.5734185863077631)
# cubrt(z/r): 0.57328 ( 3 | 08m02s |    0.57328 |    0.0084 | )
# cubrt(z/rt): 9 | 08m02s |    0.57326 |    0.0004 | (0.0003934725518432613)
# cubrt(z/rt): 0.0262 (0.5735961261910194)
# z4 0.0318 (0.5736635664313068)
# z4 0.0512 (0.5737114634140108)
# zz4 (with rt): 4 | 11m28s |    0.57371 |    0.0006 | 
# zz4 (with r):  3 | 09m28s |    0.57374 |    0.0004 | (0.00041414808716101244)
# test w12, a2 = a0 - np.nan_to_num(np.arccos(mm*ii*rt)) (0.0003: 0.5738085893381745)

0.0003
0.0003
 179.0000000[ 0] score : 0.56441303
 179.0000000[ 1] score : 0.57253485
 179.0000000[ 2] score : 0.57464970
 179.0000000[ 3] score : 0.57501309
 179.0000000[ 4] score : 0.57604711
 179.0000000[ 5] score : 0.57565233
 179.0000000[ 6] score : 0.57638302
 179.0000000[ 7] score : 0.57577558
--------------------------------------
0.5738085893381745
0.00035
0.00035
 179.0000000[ 0] score : 0.56431711
 179.0000000[ 1] score : 0.57243893
 179.0000000[ 2] score : 0.57455378
 179.0000000[ 3] score : 0.57491717
 179.0000000[ 4] score : 0.57595120
 179.0000000[ 5] score : 0.57555641
 179.0000000[ 6] score : 0.57628711
 179.0000000[ 7] score : 0.57567967
--------------------------------------
0.573712672017766
0.0004
0.0004
 179.0000000[ 0] score : 0.56435022
 179.0000000[ 1] score : 0.57242056
 179.0000000[ 2] score : 0.57453541
 179.0000000[ 3] score : 0.57489880
 179.0000000[ 4] score : 0.57593282
 179.0000000[ 5] score : 0.57553804
 179.0000000[ 6] score : 0.57626873
 179.0000000[

In [ ]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 5, n_iter = 10, acq = "ucb", kappa = 2.576)
    

In [11]:
def do_dbscan_ensemble_hck(truth):
    
    #--------------------------------------------------------
    df = truth.copy()
    df = df.assign(r   = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(d   = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(a   = np.arctan2(df.y, df.x))
    df = df.assign(cosa= np.cos(df.a))
    df = df.assign(sina= np.sin(df.a))
    df = df.assign(phi = np.arctan2(df.z, df.r))
    df = df.assign(momentum = np.sqrt( df.px**2 + df.py**2 + df.pz**2 ))
    df.loc[df.particle_id==0,'momentum']=0

    #df = df.loc[df.z>500] # consider dataset subset
    #df = df.loc[df.r<50 ] ## 0.04397/0.04750  (0.92569)

#     df = df.loc[df.z>500]
#     df = df.loc[(df.r>50) & (df.r<100)] ## 0.05259/0.05808  (0.90551)
    
    N = len(df)

    layer_id = df['layer_id'].values.astype(np.float32)
    momentum = df['momentum'].values.astype(np.float32)
    p = df[['particle_id']].values.astype(np.int64)
    x,y,z,r,a,cosa,sina,phi = df[['x', 'y', 'z', 'r', 'a', 'cosa', 'sina', 'phi']].values.astype(np.float32).T

    particle_ids = np.unique(p)
    particle_ids = particle_ids[particle_ids!=0]
    num_particle_ids = len(particle_ids)
    
    dj=0
    di=0
    EPS=1e-12
    #if 1:

    candidates = []
    for dj in np.arange(-20, 20+EPS, 10):
        for di in np.arange(-0.003, 0.003+EPS, 0.00025):
            ar = a +  di*r
            zr = (z+dj)/r  *0.1
            data2 = np.column_stack([ar,zr])

            _,l = dbscan(data2, eps=0.0025, min_samples=1,)
            track_ids = np.unique(l)
            track_ids = track_ids[track_ids!=0]
            neigbour = [ np.where(l==t)[0] for t in track_ids]

            unique,inverse,c = np.unique(l,return_counts=True,return_inverse=True)
            unique = unique[unique!=0]
            c = c[inverse]
            c[l==0] = 0

            for u in unique:
                candidate = np.where(l==u)[0]
                candidates.append(candidate)

    #---
    #<todo>
    #fix angle discontinunity problem here ...


    #-----
    #sort
    count = np.array([len(candidate) for candidate in candidates])
    sort = np.argsort(-count)
    candidates = [candidates[s] for s in sort]


    #show
    max_label=1
    label = np.zeros(N,np.int32)
    count = np.zeros(N,np.int32)

    for candidate in candidates:
        n = candidate
        L = len(n)
        #print(L)


        #---- filtering (secret sauce) ----------
        #if L<3: continue
        n = n[np.argsort(np.fabs(z[n]))]

        layer_id0 = layer_id[n[:-1]]
        layer_id1 = layer_id[n[1: ]]
        ld = layer_id1-layer_id0
        if np.any(ld>2): continue

        m = count[n].max()
        if L<m: continue

        #---- filtering ----------------------

        count[n]=L
        label[n]=max_label
        max_label += 1

    ##-###################################################################################3
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(df), df.hit_id.values, label))
    ).astype(int)
    score1 = score_event(df, submission)
    

    #print results
    max_score = df.weight.sum()
    print('max_score = df.weight.sum() = %0.5f'%max_score)
    print('score1= %0.5f  (%0.5f)'%(score1*max_score,score1))
    
#     df = df.assign(r   = np.sqrt( df.x**2 + df.y**2))
#     df = df.assign(d   = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
#     df = df.assign(a   = np.arctan2(df.y, df.x))
#     df = df.assign(cosa= np.cos(df.a))
#     df = df.assign(sina= np.sin(df.a))
#     df = df.assign(phi = np.arctan2(df.z, df.r))
#     df = df.assign(momentum = np.sqrt( df.px**2 + df.py**2 + df.pz**2 ))
    
    df = df.drop(['r','d','a','cosa','sina','phi','momentum'], axis=1 )
    return df, label

In [12]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##
]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
#     truth = truth.merge(particles,  on=['particle_id'], how='left')
    
    hits1, l1 = do_dbscan_ensemble_hck(hits)
    
    hits1['track_id'] = l1
    
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits1), hits1.hit_id.values, hits1['track_id'].values))
    ).astype(int)
    
    score = score_event(hits1, submission)
    i = 0
    print('[%2d] score : %0.8f'%(i, score))
        
    hits2 = hits[~hits.hit_id.isin(hits1.hit_id)]
#     w1 = [0.9, 1.2]
#     w2 = [0.3, 0.7]
#     w3 = [0.1, 0.4]
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.8]
    w3 = [0.1, 0.4]
    w4 = [0.1, 0.8]
    
    Niter = [140, 190]
    
    l2 = do_dbscan_predict(hits2)
    hits2['track_id'] = l2
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits2), hits2.hit_id.values, hits2['track_id'].values))
    ).astype(int)
    
    score = score_event(hits2, submission)
    i = 1
    print('[%2d] score : %0.8f'%(i, score))
     
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits2), hits2.hit_id.values, hits2['track_id'].values))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits2)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    
    print('--------------------------------------')
    print(sum_score/sum)

    hits = hits1.append(hits2)
    
    hits.sort_values('hit_id', inplace=True)
    
    track_id = hits['track_id'].values
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    score = score_event(hits, submission)
    i = 2
    print('[%2d] score : %0.8f'%(i, score))
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'w4':w4,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    

max_score = df.weight.sum() = 1.00000
score1= 0.24221  (0.24221)
[ 0] score : 0.24221202


ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by StandardScaler.

In [ ]:
hits = hits1.append(hits2)
hits.sort_values('hit_id', inplace=True)
    
track_id = hits['track_id'].values
hits['event_id'] = 1030
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
).astype(int)
print(submission.shape)
print(hits.shape)
for i in range(8):
    if i > 0:
        var1 = 1
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
print(sum_score/sum)

In [69]:
submission.head()

,event_id,hit_id,track_id
0,1030,1,0
1,1030,2,16907663
2,1030,3,8411454
3,1030,4,24229213
4,1030,5,16751177


In [68]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,weight,vx,vy,vz,px,py,pz,q,nhits,track_id
0,1,-60.8722,-2.90994,-1502.5,7,2,1,427858663433043968,-60.8900,-2.93122,...,0.000008,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,0
1,2,-68.6209,-6.93030,-1502.5,7,2,1,923241222145835008,-68.6150,-6.94011,...,0.000008,0.012087,-0.023069,13.10000,-0.216069,-0.041984,-4.86522,1.0,13.0,16907663
2,3,-64.5802,2.66632,-1502.5,7,2,1,4523734434054144,-64.5634,2.67475,...,0.000006,-0.009604,0.008012,-2.28776,-0.292098,0.031685,-6.75259,-1.0,16.0,8411454
3,4,-88.7343,-7.35970,-1502.5,7,2,1,261225408500858880,-88.7490,-7.38212,...,0.000012,0.001303,0.017286,-8.23350,-0.894095,-0.099110,-15.15630,1.0,14.0,24229213
4,5,-88.3262,-13.04520,-1502.5,7,2,1,743099023757410304,-88.3163,-13.05070,...,0.000006,-0.011713,-0.011290,-5.07843,-0.283424,-0.068196,-4.91150,1.0,16.0,16751177


In [53]:
hits.shape

(113998, 24)

In [54]:
hits.T

,0,1,2,3,4,5,6,7,8,9,...,113988,113989,113990,113991,113992,113993,113994,113995,113996,113997
hit_id,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,8.000000e+00,9.000000e+00,1.000000e+01,...,1.139890e+05,1.139900e+05,1.139910e+05,1.139920e+05,1.139930e+05,1.139940e+05,1.139950e+05,1.139960e+05,1.139970e+05,1.139980e+05
x,-6.087220e+01,-6.862090e+01,-6.458020e+01,-8.873430e+01,-8.832620e+01,-6.308520e+01,-9.462820e+01,-7.685770e+01,-7.033050e+01,-7.952860e+01,...,-7.833430e+02,-8.502360e+02,-9.015590e+02,-8.467240e+02,-9.298330e+02,-9.431250e+02,-9.615630e+02,-9.594230e+02,-9.390410e+02,-9.010720e+02
y,-2.909940e+00,-6.930300e+00,2.666320e+00,-7.359700e+00,-1.304520e+01,8.780960e-01,-1.128420e+01,-3.215100e+00,-1.388590e+01,-1.315530e+01,...,7.238810e+01,5.880680e+00,1.790310e+01,5.946570e+01,5.735790e+01,1.172240e+01,4.991550e+01,8.392830e+01,7.663410e+01,1.761350e+01
z,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,-1.502500e+03,...,2.944500e+03,2.944500e+03,2.944500e+03,2.944500e+03,2.952500e+03,2.952500e+03,2.952500e+03,2.952500e+03,2.952500e+03,2.952500e+03
volume_id,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,...,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01
layer_id,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,...,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01
module_id,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,9.800000e+01,9.800000e+01,9.800000e+01,9.800000e+01,9.800000e+01,9.800000e+01
particle_id,4.278587e+17,9.232412e+17,4.523734e+15,2.612254e+17,7.430990e+17,1.531238e+17,1.306098e+17,8.647023e+17,5.224192e+17,7.881326e+17,...,9.998082e+17,9.457585e+17,8.109750e+16,1.216090e+17,0.000000e+00,0.000000e+00,8.196566e+17,6.214984e+17,5.854803e+16,8.109750e+16
tx,-6.089000e+01,-6.861500e+01,-6.456340e+01,-8.874900e+01,-8.831630e+01,-6.307700e+01,-9.462610e+01,-7.684700e+01,-7.034720e+01,-7.954760e+01,...,-7.795640e+02,-8.548940e+02,-9.019290e+02,-8.487420e+02,-9.256440e+02,-9.446320e+02,-9.649620e+02,-9.620440e+02,-9.386240e+02,-9.043060e+02
ty,-2.931220e+00,-6.940110e+00,2.674750e+00,-7.382120e+00,-1.305070e+01,8.722560e-01,-1.129730e+01,-3.193100e+00,-1.387380e+01,-1.317410e+01,...,7.211010e+01,6.184500e+00,1.797050e+01,5.956670e+01,5.711410e+01,1.176020e+01,5.014110e+01,8.408200e+01,7.659500e+01,1.780720e+01


In [ ]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
184.7827709130419
184
1.0561046005761139 0.5351622558071004 0.3762123539827661 184
 179.0000000[ 0] score : 0.53942962
 179.0000000[ 1] score : 0.54606243
 179.0000000[ 2] score : 0.54780086
 179.0000000[ 3] score : 0.54875295
 179.0000000[ 4] score : 0.54815666
 179.0000000[ 5] score : 0.54860441
 179.0000000[ 6] score : 0.54801785
 179.0000000[ 7] score : 0.54881484
--------------------------------------
0.5469549540012781
    1 | 05m08s |    0.54695 |  184.7828 |    1.0561 |    0.5352 |    0.3762 | 
145.0794227877262
145
1.1353520277193074 0.658195945930628 0.24032322705809864 145
 179.0000000[ 0] score : 0.52382250
 179.0000000[ 1] score : 0.52837456
 179.0000000[ 2] score : 0.53034994
 179.0000000[ 3] score : 0.53139514
 179.0000000[ 4] score : 0.53153080
 179.0000000[ 5] score : 0.53243534
 179.0000000[ 6] scor

 179.0000000[ 2] score : 0.45956269
 179.0000000[ 3] score : 0.46153693
 179.0000000[ 4] score : 0.46225305
 179.0000000[ 5] score : 0.46312552
 179.0000000[ 6] score : 0.46227716
 179.0000000[ 7] score : 0.46323152
--------------------------------------
0.46055974873573446
   17 | 05m21s |    0.46056 |  186.4388 |    1.2000 |    0.3000 |    0.1000 | 
175.93802446513746
175
1.2 0.7 0.4 175
 179.0000000[ 0] score : 0.52847339
 179.0000000[ 1] score : 0.53424707
 179.0000000[ 2] score : 0.53617756
 179.0000000[ 3] score : 0.53704935
 179.0000000[ 4] score : 0.53663841
 179.0000000[ 5] score : 0.53773147
 179.0000000[ 6] score : 0.53728202
 179.0000000[ 7] score : 0.53720027
--------------------------------------
0.5355999427729057
   18 | 05m21s |    0.53560 |  175.9380 |    1.2000 |    0.7000 |    0.4000 | 
164.75000708655898
164
0.9 0.7 0.4 164
 179.0000000[ 0] score : 0.52636347
 179.0000000[ 1] score : 0.53308124
 179.0000000[ 2] score : 0.53533614
 179.0000000[ 3] score : 0.53587774

In [96]:
w1 = 1.0561046005761139
w2 = 0.5351622558071004
w3 = 0.3762123539827661
Niter = 184

# 172.3593 |    1.0579 |    0.5147 |    0.3853

In [103]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
# w4 = [0.1, 0.2] # 0.1470 -> 0.55690
w4 = 0.1470
w5 = [0.01, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
w5 = 0.7781
Niter = 247

In [104]:
def run_make_submission():

    data_dir = '../data/test'


    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)

#             track_id = trackML2(hits, w1, w2, w3, Niter)
            track_id = trackML3(hits, w1,w2,w3,w4,w5,Niter)
            #track_id = back_fit(track_id,hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, track_id))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub2/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub2/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub2/submission-0020.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [105]:
run_make_submission()

event_id : 000000000  , 4 min
event_id : 000000001  , 11 min
event_id : 000000002  , 17 min
event_id : 000000003  , 23 min
event_id : 000000004  , 29 min
event_id : 000000005  , 36 min
event_id : 000000006  , 42 min
event_id : 000000007  , 48 min
event_id : 000000008  , 54 min
event_id : 000000009  , 61 min
event_id : 000000010  , 67 min
event_id : 000000011  , 73 min
event_id : 000000012  , 79 min
event_id : 000000013  , 86 min
event_id : 000000014  , 92 min
event_id : 000000015  , 97 min
event_id : 000000016  , 102 min
event_id : 000000017  , 109 min
event_id : 000000018  , 115 min
event_id : 000000019  , 120 min
event_id : 000000020  , 125 min
event_id : 000000021  , 131 min
event_id : 000000022  , 137 min
event_id : 000000023  , 143 min
event_id : 000000024  , 150 min
event_id : 000000025  , 156 min
event_id : 000000026  , 162 min
event_id : 000000027  , 168 min
event_id : 000000028  , 174 min
event_id : 000000029  , 180 min
event_id : 000000030  , 186 min
event_id : 000000031  , 1